In [1]:
path = '/Users/longbro/Downloads/小贞毕设/'
import numpy as np
np.linspace(1,10)

array([  1.        ,   1.18367347,   1.36734694,   1.55102041,
         1.73469388,   1.91836735,   2.10204082,   2.28571429,
         2.46938776,   2.65306122,   2.83673469,   3.02040816,
         3.20408163,   3.3877551 ,   3.57142857,   3.75510204,
         3.93877551,   4.12244898,   4.30612245,   4.48979592,
         4.67346939,   4.85714286,   5.04081633,   5.2244898 ,
         5.40816327,   5.59183673,   5.7755102 ,   5.95918367,
         6.14285714,   6.32653061,   6.51020408,   6.69387755,
         6.87755102,   7.06122449,   7.24489796,   7.42857143,
         7.6122449 ,   7.79591837,   7.97959184,   8.16326531,
         8.34693878,   8.53061224,   8.71428571,   8.89795918,
         9.08163265,   9.26530612,   9.44897959,   9.63265306,
         9.81632653,  10.        ])

In [2]:
# 读取csv
trans = spark.read.format('csv').load(path + 'trans.csv',header = True)
# 列名重命名1
oldColumns  = trans.columns
newColumns = ['account','trans_id','trans_name','trans_date','prod_id','prod_name','trans_way','amount']
# reduce 第三个参数是初始值
trans = reduce(lambda trans2, idx : trans2.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)),trans)

# 列名重命名2
from pyspark.sql.functions import *
# cost = cost.select(col("客户号").alias("account"),col("产品代码").alias("prod_id"),col("投资成本").alias("amount"))

# 列重命名 3
# customer = customer.toDF('account','cus_type','cus_group','certification','gender','risk','birthday','manager')
trans.toPandas().head()

,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1003089018,100541,智能定投开通,20161017,020003,国泰金龙行业,网上银行,1000.0
1,1001172983,100003,银行账户变更,20161017,,None,网上银行,0.0
2,1001232639,100208,定期定额开通,20161017,100056,富国低碳环保,网上银行,100.0
3,1001232639,100208,定期定额开通,20161017,001616,嘉实环保低碳,网上银行,100.0
4,1003275159,100541,智能定投开通,20161017,000136,民生策略精选混合,网上银行,400.0


In [3]:
# 注册为table1
# from pyspark.sql import SQLContext
# sqlCtx = SQLContext(sc)
# sqlCtx.registerDataFrameAsTable(cost,'cost')

# 注册为table2
trans.registerTempTable('trans')
# product.registerTempTable('product')
# customer.registerTempTable('customer')

In [4]:
# 时间窗格操作  - 还有 ROLLUP、CUBE、GROUPING SETS、GROUPING__ID 可以查，主要是扩充分组的维度（年、月、日...）
# 各种组合起来
spark.sql("select account, trans_id, prod_id, amount,trans_date,  " + 
          " sum(amount) OVER(partition by account order by prod_id desc " + 
          " rows between unbounded preceding and current row) sum_account,  "+
          " row_number() OVER(partition by account order by prod_id desc) row_nums, " + 
          " rank() OVER(partition by account order by prod_id desc) rank, " + 
          " dense_rank() OVER(partition by account order by prod_id desc) dense_rank,  " + 
          " ntile(2) OVER(partition by account order by prod_id desc) ntile,   " + 
          " round(cume_dist() OVER(partition by account order by prod_id desc),2) cume_dist,  " + 
          " lag(trans_date,1 , 'filled') OVER(partition by account order by trans_date) lag_1,  " + 
          " lead(trans_date,3 , 'filled') OVER(partition by account order by trans_date) lead_3,  " + 
          " first(prod_id) OVER(partition by account order by trans_date ) first_value,  " + 
          " last(prod_id) OVER(partition by account order by trans_date ) last_value  " + 
          " from trans").toPandas().head(5)

,account,trans_id,prod_id,amount,trans_date,sum_account,row_nums,rank,dense_rank,ntile,cume_dist,lag_1,lead_3,first_value,last_value
0,1000273675,200208,340008,2000.0,20161101,2000,1,1,1,1,0.54,filled,20161102,340008,340006
1,1000273675,200208,340006,2000.0,20161101,14000,8,8,2,2,1.00,20161101,20161102,340008,340006
2,1000273675,100209,340008,2000.0,20161102,4000,2,1,1,1,0.54,20161101,20161103,340008,340006
3,1000273675,100209,340006,2000.0,20161102,16000,9,8,2,2,1.00,20161102,20161103,340008,340006
4,1000273675,100209,340006,2000.0,20161102,18000,10,8,2,2,1.00,20161102,20161124,340008,340006


In [13]:
print spark.sql("select * from trans TABLESAMPLE(10 Percent) s").toPandas().shape
print spark.sql('''with tab1 as (
select *,cast(rand()*100 as int) as rand_index from trans
)
select * from tab1
where rand_index <= 10''').toPandas().shape

print spark.sql('''with tab1 as (
select *, row_number() OVER(partition by account order by amount) as row_nums from trans
),tab2 as (
select account , max(row_nums) max_nums from tab1
group by account)
select t1.* from tab1 as t1 left join tab2 as t2
on t1.account = t2.account
where t1.row_nums/t2.max_nums < 0.5
''').toPandas().shape

(4055, 8)
(4320, 9)
(12662, 9)


In [21]:
print spark.sql('''(select * from trans TABLESAMPLE(10 Percent) s
where trans_way = "网上银行")
union
(select * from trans TABLESAMPLE(80 Percent) s2
where trans_way != "网上银行")
''').toPandas().shape


print spark.sql('''(select * from trans TABLESAMPLE(10 Percent) s
where trans_way = "网上银行")
union
(select * from trans TABLESAMPLE(80 Percent) s2
where trans_way != "网上银行")
''').toPandas().shape

(8835, 8)


In [45]:
spark.sql("select cast(trans_date as string) from trans limit 5").toPandas()
# cast(trans_date as string)
# date_add(to_date(trans_date,'yyyyMMdd'),2)
df1 = spark.sql("select trans_id, from_unixtime(unix_timestamp(trans_date,'yyyyMMdd')) as t1 from trans limit 5")
df2 = spark.sql("select trans_id, from_unixtime(unix_timestamp(trans_date,'yyyyMMdd') + 3600*24*5) as t2 from trans limit 5")
df1.registerTempTable('df1')
df2.registerTempTable('df2')
spark.sql("select datediff((cast(t2 as string)),from_unixtime(unix_timestamp('2015.12.31','yyyy.MM.dd')))/30 from df1 join df2 on df1.trans_id = df2.trans_id").toPandas()

# spark.sql("select datediff(substring(cast(t2 as string),0,10),substring(cast(t1 as string),0,10)) from df1 join df2 on df1.trans_id = df2.trans_id").toPandas()

,"(CAST(datediff(CAST(CAST(t2 AS STRING) AS DATE), CAST(from_unixtime(unix_timestamp(2015.12.31, yyyy.MM.dd), yyyy-MM-dd HH:mm:ss) AS DATE)) AS DOUBLE) / CAST(30 AS DOUBLE))"
0,9.866667
1,9.866667
2,9.866667
3,9.866667
4,9.866667
5,9.866667
6,9.866667
7,9.866667
8,9.866667


In [9]:
spark.sql("select percentile(amount,0.55) from trans").show()

+--------------------------------------------------------+
|percentile(CAST(amount AS DOUBLE), CAST(0.55 AS DOUBLE))|
+--------------------------------------------------------+
|                                                   400.0|
+--------------------------------------------------------+



In [53]:
spark.sql("select from_unixtime(unix_timestamp('2015.12.31','yyyy.MM.dd'))").show()

# spark.sql("select to_date('2015.12.31','yyyy.MM.dd')").show()
# 2015-12-31 00:00:00
spark.sql("select to_date('2015-12-31 00:00:00')").show()


+--------------------------------------------------------------------------+
|from_unixtime(unix_timestamp(2015.12.31, yyyy.MM.dd), yyyy-MM-dd HH:mm:ss)|
+--------------------------------------------------------------------------+
|                                                       2015-12-31 00:00:00|
+--------------------------------------------------------------------------+

+------------------------------------------+
|to_date(CAST(2015-12-31 00:00:00 AS DATE))|
+------------------------------------------+
|                                2015-12-31|
+------------------------------------------+



In [34]:
# CTE Common Table Expression - 另一种子查询方式，看着更整齐一些
spark.sql("with q1 as (select * from trans where account = '1000273675')  select * from q1").toPandas().head(1)

,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1000273675,200208,定期定额处理,20161101,340006,兴全全球视野,手机银行,2000.0


In [35]:
# array_contains
spark.sql("select * from trans where array_contains(split('1000273675,1000482458,1000527030' , ',') ,  account)").toPandas().head()

,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1000527030,200208,定期定额处理,20161017,161613,融通创业板指数前端,网上银行,600.0
1,1000482458,200208,定期定额处理,20161017,320006,诺安灵活配置,网上银行,400.0
2,1000273675,200208,定期定额处理,20161101,340006,兴全全球视野,手机银行,2000.0
3,1000273675,200208,定期定额处理,20161101,340008,兴全有机增长,手机银行,2000.0
4,1000273675,100209,定期定额修改,20161102,340006,兴全全球视野,网上银行,2000.0


In [36]:
# 转化日期时间为 时间戳    unix_timestamp(time ,  format)
spark.sql("select unix_timestamp(trans_date,'yyyyMMdd') from trans limit 1").show()

# 转化时间戳为日期时间     from_unixtime(time , format)
spark.sql("select from_unixtime(unix_timestamp(trans_date,'yyyyMMdd'), 'yyyy-MM-dd HH:mm:ss') from trans limit 1").show()

# year, month, day, weekofyear  方法  yyyy-MM-dd HH:mm:ss    yyyy-MM-dd
# hour, minute, second, to_date, date_add, date_sub, datediff 方法 yyyy-MM-dd
spark.sql("select month('2011-12-08 10:03:01')").show()

+------------------------------------+
|unix_timestamp(trans_date, yyyyMMdd)|
+------------------------------------+
|                          1476633600|
+------------------------------------+

+------------------------------------------------------------------------+
|from_unixtime(unix_timestamp(trans_date, yyyyMMdd), yyyy-MM-dd HH:mm:ss)|
+------------------------------------------------------------------------+
|                                                     2016-10-17 00:00:00|
+------------------------------------------------------------------------+

+----------------------------------------+
|month(CAST(2011-12-08 10:03:01 AS DATE))|
+----------------------------------------+
|                                      12|
+----------------------------------------+



In [23]:
# regexp_replace 按照正则替换字符串
# regexp_replace(string A, string B, string C)
spark.sql(''' select regexp_replace(trans_name, substring(cast(trans_name as string),1,2), '傻瓜') reg_rep from trans where account = '1003089018' ''').toPandas()

# spark.sql(''' select replace(trans_name,'智能','傻瓜') from trans''').toPandas()



# spark.sql(''' select  replace(trans_name,substring(cast(trans_name as string),1,2),'12') from trans''').toPandas()

,reg_rep
0,傻瓜定投开通
1,傻瓜定额处理
2,傻瓜定额处理


In [38]:
# regexp_extract 按照规则拆分字符串,选择需要的index,index含义是第几个括号，0为匹配的所有
# regexp_extract(string subject, string pattern, int index)
print spark.sql(''' select * from trans where account = '1003089018' and regexp_extract(trans_name, '智能', 0) = '' ''').toPandas()

spark.sql(''' select * from trans where account = '1003089018' and regexp_extract(trans_name, '智能', 0) != '' ''').toPandas()

      account trans_id trans_name trans_date prod_id prod_name trans_way  \
0  1003089018   200208     定期定额处理   20161117  020003    国泰金龙行业      网上银行   
1  1003089018   200208     定期定额处理   20161219  020003    国泰金龙行业      网上银行   

   amount  
0  1000.0  
1  1000.0  


,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1003089018,100541,智能定投开通,20161017,020003,国泰金龙行业,网上银行,1000.0


In [54]:
spark.sql('''select
regexp_extract('x=a3&x=18abc&x=2&y=3&x=4','x=([0-9]+)([a-z]+)',0),  
regexp_extract('x=a3&x=18abc&x=2&y=3&x=4','^x=([a-z]+)([0-9]+)',0),
 
regexp_extract('https://detail.tmall.com/item.htm?spm=608.7065813.ne.1.Ni3rsN&id=&tracelog=fromnonactive','([0-9]+)',1), 
regexp_extract('https://detail.tmall.com/item.htm?spm=608.7065813.ne.1.Ni3rsN&id=522228774076&tracelog=fromnonactive','id=([0-9]+)',1), 
 
regexp_extract('http://a.m.taobao.com/i41915173660.htm','i([0-9]+)',0), 
regexp_extract('http://a.m.taobao.com/i41915173660.htm','i([0-9]+)',1) 
''').toPandas()

,"regexp_extract(x=a3&x=18abc&x=2&y=3&x=4, x=([0-9]+)([a-z]+), 0)","regexp_extract(x=a3&x=18abc&x=2&y=3&x=4, ^x=([a-z]+)([0-9]+), 0)","regexp_extract(https://detail.tmall.com/item.htm?spm=608.7065813.ne.1.Ni3rsN&id=&tracelog=fromnonactive, ([0-9]+), 1)","regexp_extract(https://detail.tmall.com/item.htm?spm=608.7065813.ne.1.Ni3rsN&id=522228774076&tracelog=fromnonactive, id=([0-9]+), 1)","regexp_extract(http://a.m.taobao.com/i41915173660.htm, i([0-9]+), 0)","regexp_extract(http://a.m.taobao.com/i41915173660.htm, i([0-9]+), 1)"
0,x=18abc,x=a3,608,522228774076,i41915173660,41915173660


In [49]:
from sklearn.metrics import classification_report
?classification_report()

In [104]:
# 同账户下，过去24h购买产品的个数
# key 是交易id和交易时间的组合
spark.sql('''with cum as (select  trans_id, account , unix_timestamp(trans_date,'yyyyMMdd')  unix_trans_time  from  trans)
            select  c1.trans_id ,c1.unix_trans_time, count(*) from cum c1 left outer join cum c2 on c1.account = c2.account
            and c1.unix_trans_time -  c2.unix_trans_time > 24*60*60 
            where c1.account = '1003089018' 
            group by c1.trans_id, c1.unix_trans_time ''').toPandas()

,trans_id,unix_trans_time,count(1)
0,100541,1476633600,1
1,200208,1479312000,1
2,200208,1482076800,2


In [8]:
# 同账户下，购买产品的时间间隔的标准差
# key 是账户
spark.sql('''with cum as (select  account , unix_timestamp(trans_date,'yyyyMMdd') unix_trans_time , 
            unix_timestamp(lag(trans_date, 1 , '20100101') over(partition by account order by trans_date),'yyyyMMdd') unix_lag_time from  trans)
            select account,  round(std(unix_lag_time - unix_trans_time)/3600,2)
            from cum group by account''').toPandas().head()

,account,"round((stddev_samp(CAST((unix_lag_time - unix_trans_time) AS DOUBLE)) / CAST(3600 AS DOUBLE)), 2)"
0,1000273675,16581.15
1,1000283935,42120.94
2,1000482458,24156.30
3,1000527030,29520.90
4,1000558961,NaN


In [9]:
# 同账户下，历史最大交易金额 和 当前交易金额的 比例
spark.sql('''with cum as (select  account , row_number() over(partition by account order by amount desc) row_nums, amount from trans)
        select trans.trans_id, trans.account, round(trans.amount / cum.amount, 2) now_max_rate
        from trans left join cum on cum.row_nums = 1 and trans.account = cum.account
        where round(trans.amount / cum.amount, 2) is not null''').toPandas().head()
#         ''').show()
# .toPandas().head()
        

,trans_id,account,now_max_rate
0,100541,1003089018,1
1,100208,1001232639,1
2,100208,1001232639,1
3,100541,1003275159,1
4,100208,1003276023,1


In [82]:
# 空判断
# 1. is null  "\N"     2. = ""

In [9]:
trans.toPandas().head()

,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1003089018,100541,智能定投开通,20161017,020003,国泰金龙行业,网上银行,1000.0
1,1001172983,100003,银行账户变更,20161017,,None,网上银行,0.0
2,1001232639,100208,定期定额开通,20161017,100056,富国低碳环保,网上银行,100.0
3,1001232639,100208,定期定额开通,20161017,001616,嘉实环保低碳,网上银行,100.0
4,1003275159,100541,智能定投开通,20161017,000136,民生策略精选混合,网上银行,400.0


In [15]:
spark.sql('''select trans_id, (case when amount -prod_id > 100 then 'yes' else 'no' end) amountover100 from trans
''').toPandas().head()

spark.sql('''select trans_id, (case when amount -prod_id > 100 then amount else prod_id end) amountover100 from trans
''').toPandas().head()

# spark.sql('''select trans_id, (case when amount > 100 then prod_name else 'no' end) amountover100 from trans
# ''').toPandas().head()

,trans_id,amountover100
0,100541,020003
1,100003,
2,100208,100056
3,100208,001616
4,100541,400.0


In [60]:
from pyspark.sql import functions as F

In [ ]:
trans.

In [12]:
spark.sql("select * from trans where array_contains(split('1000273675,1000482458,1000527030' , ',') ,  account)").toPandas().head()

,account,trans_id,trans_name,trans_date,prod_id,prod_name,trans_way,amount
0,1000527030,200208,定期定额处理,20161017,161613,融通创业板指数前端,网上银行,600.0
1,1000482458,200208,定期定额处理,20161017,320006,诺安灵活配置,网上银行,400.0
2,1000273675,200208,定期定额处理,20161101,340006,兴全全球视野,手机银行,2000.0
3,1000273675,200208,定期定额处理,20161101,340008,兴全有机增长,手机银行,2000.0
4,1000273675,100209,定期定额修改,20161102,340006,兴全全球视野,网上银行,2000.0


In [15]:
spark.sql("select count(*) from trans").toPandas().head()

,count(1)
0,39636


In [17]:
spark.sql('''select * from (select account,trans_date,amount from trans order by account,trans_date) as a
          left outer join (select account,trans_date,amount from trans order by account,trans_date) as b on a.account=b.account and a.tr ''').toPandas().head()

,account,trans_date,amount,account,trans_date,amount
0,1000000132,20161017,500.0,1000000132,20170103,500.0
1,1000000132,20161017,500.0,1000000132,20170103,500.0
2,1000000132,20161017,500.0,1000000132,20161219,500.0
3,1000000132,20161017,500.0,1000000132,20161219,500.0
4,1000000132,20161017,500.0,1000000132,20161219,500.0
